In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression,RidgeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
import mediapipe as mp
import cv2

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        
        # Make Detections
        results = holistic.process(image)
        
        
       
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks,mp_holistic.FACEMESH_TESSELATION , 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
    
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
right_hand_landmarks_count = 0
for landmark in results.right_hand_landmarks.landmark:
        right_hand_landmarks_count += 1
right_hand_landmarks_count

In [ ]:
left_hand_landmarks_count = 0
for landmark in results.left_hand_landmarks.landmark:
        left_hand_landmarks_count += 1
left_hand_landmarks_count

# creating file for inputs or for storing coordinate

In [ ]:
import csv 
import os

In [ ]:
i=len(results.face_landmarks.landmark)
j=len(results.pose_landmarks.landmark)
k = right_hand_landmarks_count
l= left_hand_landmarks_count
num_coords = len(results.face_landmarks.landmark)+len(results.pose_landmarks.landmark)+right_hand_landmarks_count+left_hand_landmarks_count
print(i,j,k,l)
print(num_coords)

In [ ]:
landmarks = ['class']
for val in range(1,num_coords + 1):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [ ]:
landmarks

In [ ]:
with open ('coord.csv',mode = 'w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar = '"',quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks) 

# take the inputs

In [ ]:
class_name = 'happy'

In [ ]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.2, min_tracking_confidence=0.2) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)

        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # ... (Drawing landmarks code)
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                   mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                   mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                   )

        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                   mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                   mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                                   )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                   mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                   mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                   )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                   mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                   mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                   )

        # Export coordinates
        try:
            # Extract landmarks for the left hand
            left_hand_row = []
            left_hand_landmarks = results.left_hand_landmarks

            if left_hand_landmarks:
                left_hand_landmark_list = [
                    (landmark.x, landmark.y, landmark.z, landmark.visibility)
                    for landmark in left_hand_landmarks.landmark
                ]
                left_hand_row = list(np.array(left_hand_landmark_list).flatten())
            else:
                # If left hand landmarks are not detected, create an empty row
                left_hand_row = [0.0] * (4 * left_hand_landmarks_count)  # Assuming 4 values per landmark

            # Extract landmarks for the right hand
            right_hand_row = []
            right_hand_landmarks = results.right_hand_landmarks

            if right_hand_landmarks:
                right_hand_landmark_list = [
                    (landmark.x, landmark.y, landmark.z, landmark.visibility)
                    for landmark in right_hand_landmarks.landmark
                ]
                right_hand_row = list(np.array(right_hand_landmark_list).flatten())
            else:
                # If right hand landmarks are not detected, create an empty row
                right_hand_row = [0.0] * (4 * right_hand_landmarks_count)  # Assuming 4 values per landmark

            # Extract face and pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            # Concatenate row
            row = pose_row + face_row + left_hand_row + right_hand_row

            # Append name
            row.insert(0, class_name)

            # Extract to CSV
            with open('coord.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except Exception as e:
            print(f"Error: {e}")

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# Train the data

In [ ]:
df= pd.read_csv('coord.csv')
df

In [ ]:
x = df.drop('class',axis=1)
y = df['class']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train,y_train)
    fit_models[algo] = model

# finding best algo for our model

In [ ]:
from sklearn.metrics import accuracy_score 
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:
fit_models['rf'].predict(x_test)

# loading the model with help of pickle

In [ ]:
import pickle 

In [ ]:
with open('your_trained_model.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)